# Foundation of Computer Science Project

Riccardo Cervero

Roll number: 794126

Academic year 2018/2019

Data Science

Department of Informatics, Systems and Communication

University of Milano Bicocca

r.cervero@campus.unimib.it

Import necessary libraries: re, pandas, numpy and time

In [5]:
import pandas as pd
import numpy as np
import re
import time

In [6]:
start = time.time()

Upload database using Pandas

In [7]:
playstore = pd.read_csv('/users/riccardocervero/Desktop/googleplaystore.csv')
playstore_users = pd.read_csv('/users/riccardocervero/Desktop/googleplaystore_user_reviews.csv')

View 'playstore' data

In [8]:
playstore.head(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


In [9]:
set(playstore['Category'])

{'1.9',
 'ART_AND_DESIGN',
 'AUTO_AND_VEHICLES',
 'BEAUTY',
 'BOOKS_AND_REFERENCE',
 'BUSINESS',
 'COMICS',
 'COMMUNICATION',
 'DATING',
 'EDUCATION',
 'ENTERTAINMENT',
 'EVENTS',
 'FAMILY',
 'FINANCE',
 'FOOD_AND_DRINK',
 'GAME',
 'HEALTH_AND_FITNESS',
 'HOUSE_AND_HOME',
 'LIBRARIES_AND_DEMO',
 'LIFESTYLE',
 'MAPS_AND_NAVIGATION',
 'MEDICAL',
 'NEWS_AND_MAGAZINES',
 'PARENTING',
 'PERSONALIZATION',
 'PHOTOGRAPHY',
 'PRODUCTIVITY',
 'SHOPPING',
 'SOCIAL',
 'SPORTS',
 'TOOLS',
 'TRAVEL_AND_LOCAL',
 'VIDEO_PLAYERS',
 'WEATHER'}

Among 'Category' values, the unexpected value '1.9' is found, due to the fact that all the elements of the row are moved to the left. Thus, '1.9' is likely a 'Rating' value.

In [10]:
playstore[playstore['Category']=='1.9']

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


Row number 10472 is the only one containing this unexpected value. All values should be moved to the right, but unfortunately we don't have enough information to know whether 'Category' value should be set equal to 'PhotoFrame' or equal to None - in case the name of the Application was 'Life Made WI-Fi Touchscreen Photo Frame'. Thus, the best solution is to remove the entire row number 10472.

In [11]:
playstore = playstore.drop(index=10472)

View 'playstore_users' data

In [12]:
playstore_users.head(3)

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN


# **1) Convert the app sizes to a number**

In [13]:
#Compile, group and name patterns
to_int_re = re.compile('(?P<amount>\d+\.{0,1}\d*)(?P<unit>\w{0,1})')
#Define a function which transform Giga, Mega and Kilo in numbers
def unit_to_mult(unit):
    if unit == 'G':
        return 1000000000
    if unit == 'M':
        return 1000000
    if unit == 'K':
        return 1000
    return 1
#Define a function which definitely trasform the size in value in an integer 
def to_num(elem):
    m = to_int_re.search(elem)
    if m is None:
        return np.nan 
    unit = m.group('unit')
    mult = unit_to_mult(unit)
    amount = float(m.group('amount'))
    return int(amount * mult)
#Apply function to the column
playstore['Size'] = playstore['Size'].apply(to_num)
playstore.head(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000.0,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


# **2) Convert the number of installs to a number**

In [14]:
#Compile, group and name patterns
n_to_i = re.compile('(?P<digit>[\d*,]*)')
#Define a function which transform installs in number
def digit_to_number (digit):
    searched = n_to_i.search(digit)
    if pd.isnull(searched):
        return None
    else:
        return int(searched.group('digit').replace(',',''))
#Apply function to the column
playstore['Installs'] = playstore['Installs'].apply(digit_to_number)
playstore.head(1)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up


# **3) Transform “Varies with device” into a missing value**

In [15]:
playstore.replace('Varies with device',np.nan,inplace=True)

Parameter 'inplace' is set equal to True in order to make the variation on all the views

# **4) Convert Current Ver and Android Ver into a dotted number**

In [16]:
playstore['Current Ver'].head(10)

0       1.0.0
1       2.0.0
2       1.2.4
3         NaN
4         1.1
5         1.0
6         1.1
7    6.1.61.1
8       2.9.2
9         2.8
Name: Current Ver, dtype: object

In [17]:
playstore['Android Ver'].head(10)

0    4.0.3 and up
1    4.0.3 and up
2    4.0.3 and up
3      4.2 and up
4      4.4 and up
5      2.3 and up
6    4.0.3 and up
7      4.2 and up
8      3.0 and up
9    4.0.3 and up
Name: Android Ver, dtype: object

In [18]:
to_dot = re.compile('(?P<digit>[\d*\.]*)')
#Define the function
def to_dotted (digit):
    found = to_dot.search(digit)
    if pd.isnull(found):
        return np.nan
    else:
        return found.group('digit')

'Android Ver' and 'Current Ver' are type 'object'. To apply the function, it is necessary to convert them to string:

In [19]:
playstore['Current Ver'] = playstore['Current Ver'].astype(str)
playstore['Android Ver'] = playstore['Android Ver'].astype(str)

Finally applying the function to the columns:

In [20]:
playstore['Current Ver'] = playstore['Current Ver'].apply(to_dotted)
playstore['Android Ver'] = playstore['Android Ver'].apply(to_dotted)

In [21]:
playstore.head(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19000000.0,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14000000.0,500000,Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8700000.0,5000000,Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3


# **5) Remove the duplicates**

First, I drop rows whose values are equal in all the columns. I decide to keep the first duplicate (keep='first') and drop the other ones directly from the databases (inplace = True)

In [22]:
playstore.drop_duplicates(keep='first',inplace=True)
playstore_users.drop_duplicates(keep='first',inplace=True)

Nevertheless, other kind of duplicates are left in the database: ones referred to the same App. It is necessary to drop these ones too. In order to do it, I decide to look at 'Reviews' column and keep, for each App, the row with highest value, because it is likely the newest version. 

First, I analyze 'Review':

In [23]:
playstore['Reviews'].dtype

dtype('O')

It is necessary to convert 'Reviews' into integer:

In [24]:
playstore['Reviews']=pd.to_numeric(playstore['Reviews'])

Then, sort the database by 'Reviews' - ascending order - so that, keeping the first, I can keep the one with highest value of 'Reviews'

In [25]:
playstore = playstore.sort_values('Reviews',ascending=False)

Let's see one example of duplicates:

In [26]:
playstore[playstore['App']=='Coloring book moana'][['App','Reviews']]

,App,Reviews
2033,Coloring book moana,974
1,Coloring book moana,967


In [27]:
playstore.drop_duplicates(subset='App',keep='first',inplace=True)
playstore[playstore['App']=='Coloring book moana'][['App','Reviews']]

,App,Reviews
2033,Coloring book moana,974


As you can see, it works!

# **6) For each category, compute the number of apps**

In [28]:
playstore.groupby('Category')['App'].count()

Category
ART_AND_DESIGN           61
AUTO_AND_VEHICLES        85
BEAUTY                   53
BOOKS_AND_REFERENCE     222
BUSINESS                420
COMICS                   56
COMMUNICATION           315
DATING                  170
EDUCATION               105
ENTERTAINMENT            86
EVENTS                   64
FAMILY                 1878
FINANCE                 345
FOOD_AND_DRINK          112
GAME                    945
HEALTH_AND_FITNESS      288
HOUSE_AND_HOME           73
LIBRARIES_AND_DEMO       84
LIFESTYLE               369
MAPS_AND_NAVIGATION     131
MEDICAL                 395
NEWS_AND_MAGAZINES      254
PARENTING                60
PERSONALIZATION         376
PHOTOGRAPHY             281
PRODUCTIVITY            374
SHOPPING                202
SOCIAL                  239
SPORTS                  325
TOOLS                   829
TRAVEL_AND_LOCAL        219
VIDEO_PLAYERS           164
WEATHER                  79
Name: App, dtype: int64

# **7) For each category, compute the average rating**

Check if 'Rating' is number or string:

In [29]:
playstore['Rating'].dtype

dtype('float64')

So we can compute the mean:

In [30]:
playstore.groupby('Category')['Rating'].mean()

Category
ART_AND_DESIGN         4.359322
AUTO_AND_VEHICLES      4.190411
BEAUTY                 4.278571
BOOKS_AND_REFERENCE    4.344970
BUSINESS               4.098479
COMICS                 4.181481
COMMUNICATION          4.121484
DATING                 3.980451
EDUCATION              4.349038
ENTERTAINMENT          4.129070
EVENTS                 4.435556
FAMILY                 4.184150
FINANCE                4.115563
FOOD_AND_DRINK         4.171277
GAME                   4.244432
HEALTH_AND_FITNESS     4.243033
HOUSE_AND_HOME         4.140984
LIBRARIES_AND_DEMO     4.178125
LIFESTYLE              4.093355
MAPS_AND_NAVIGATION    4.036441
MEDICAL                4.165862
NEWS_AND_MAGAZINES     4.121569
PARENTING              4.300000
PERSONALIZATION        4.332215
PHOTOGRAPHY            4.155894
PRODUCTIVITY           4.183389
SHOPPING               4.230556
SOCIAL                 4.247291
SPORTS                 4.216154
TOOLS                  4.040278
TRAVEL_AND_LOCAL       4.069519

# **8) Create two dataframes: one for the genres and one bridging apps and genres.** 

**So that, for instance, the app Pixel Draw - Number Art Coloring Book appears twice in the bridging table, once for Art & Design, once for Creativity**

As we notice, genres are often merge toghether,separated by ";". Thus, it is necessary to split them:

In [31]:
playstore['Splitted'] = [genre.split(';') for genre in playstore['Genres']]
Genre_splitted = playstore.Splitted.apply(pd.Series)
Genre_splitted = Genre_splitted.rename(columns = {0:'First_genre', 1:'Second_genre'})
Genre_splitted.head()

,First_genre,Second_genre
2544,Social,NaN
336,Communication,NaN
2604,Social,NaN
382,Communication,NaN
1879,Strategy,NaN


The function I applied splits the values containing ';' in two parts, and distributes these ones in two different columns. We can understand this by looking at the line num. 27:

In [32]:
Genre_splitted.iloc[[27]]

,First_genre,Second_genre
2050,Casual,Action & Adventure


The bridge table between Apps and Genre is created by using a sequence of function:
- merge() merges toghether the dataframe Genre_splitted - composed by columns 'First_genre' and 'Second_genre'- and the column 'App' from the playstore database converted to dataframe (with function to_frame)
- melt() identifies 'App' column as index in the new dataframe called 'AppsnGenres' and creates other two columns: one collecting all the values of 'Genre_splitted' in a unique column called 'value' and other one, called 'variable', indicating if the Genre belongs to 'First_genre' or 'Second_genre'
- drop('variable') eliminates the column 'variable', because I don't need it
- dropna() drops all the rows with null genre, assuming every App has a known genre

In [33]:
Apps = playstore['App']
AppsnGenres = pd.merge(Genre_splitted, Apps.to_frame(), left_index = True, right_index = True)\
.melt(id_vars = ['App'], value_name = "Unique Genre")\
.drop("variable", axis = 1)\
.dropna() #Dropping Apps with null genres, actually Apps with only one unique genre
AppsnGenres.head()

,App,Unique Genre
0,Facebook,Social
1,WhatsApp Messenger,Communication
2,Instagram,Social
3,Messenger – Text and Video Chat for Free,Communication
4,Clash of Clans,Strategy


Checking if 'Pixel Draw - Number Art Coloring Book' appears twice in the bridging table:

In [34]:
AppsnGenres[AppsnGenres['App'] == 'Pixel Draw - Number Art Coloring Book']

,App,Unique Genre
4831,Pixel Draw - Number Art Coloring Book,Art & Design
14490,Pixel Draw - Number Art Coloring Book,Creativity


In conclusion, the Genres table is easily generated:

In [35]:
Gen_tab = pd.DataFrame(AppsnGenres['Unique Genre'].unique())
Gen_tab.head()

,0
0,Social
1,Communication
2,Strategy
3,Tools
4,Arcade


# **9) For each genre, create a new column of the original dataframe. The new columns must have boolean values (True if the app has a given genre)**

I create a function such that the value of dummy variables be true if the App has the given genre: 

In [159]:
def new_col( df, genres ):
    dummies = pd.get_dummies(df[genres].apply(pd.Series).stack()).sum(level=0)
    dummies.drop(dummies.columns[0], axis=1, inplace=True) #2
    df = pd.concat([df,dummies.astype(bool)], axis=1) #3
    df.drop(genres, axis = 1, inplace = True ) #4
    return df
playstore_dummy = new_col(playstore, 'Splitted')

'dummies' is the dataframe containing all dummy variables, created by get_dummies() function from the pandas series of splitted genres. The stack function of pandas allows, in this case, to associate value 1 to the dummy column in correspondence with the line associated to the genre from whom the dummy variable has been generated. sum() counts on the level '1' generated by stack function. Then, I drop first column, because it is useless (2) , concatenate the dataframe and 'dummies' with boolean values instead of numeric (3), drop 'Splitted' dataframe because it's useless (4). 

An example:

In [169]:
playstore_dummy[playstore_dummy['App']=='Pixel Draw - Number Art Coloring Book'][['App','Genres','Art & Design','Creativity','Beauty']]

,App,Genres,Art & Design,Creativity,Beauty
4,Pixel Draw - Number Art Coloring Book,Art & Design;Creativity,True,True,False


# **10) For each genre, compute the average rating. What is the genre with highest average?**

In order to accomplish this task, it is logically more correct calculating tha average rating of the splitted genres instead of values from the originale table playstore. For this reason, I merge AppsnGenres with playstore and select only columns we are interested in:

In [161]:
G = AppsnGenres.merge(playstore, on = 'App')
G = G[['App','Genres','Unique Genre','Rating']]
G.head()

,App,Genres,Unique Genre,Rating
0,Facebook,Social,Social,4.1
1,WhatsApp Messenger,Communication,Communication,4.4
2,Instagram,Social,Social,4.5
3,Messenger – Text and Video Chat for Free,Communication,Communication,4.0
4,Clash of Clans,Strategy,Strategy,4.6


In [162]:
G.groupby('Unique Genre')[['Rating']].mean().head()

,Rating
Unique Genre,
Action,4.247697
Action & Adventure,4.288542
Adventure,4.191026
Arcade,4.277838
Art & Design,4.350000


In [163]:
print("The genre with highest average is",G.groupby('Unique Genre')['Rating'].mean().idxmax(),":",G.groupby('Unique Genre')['Rating'].mean().max())

The genre with highest average is Events : 4.435555555555556


# **11) For each app, compute the approximate income, obtain as a product of number of installs and price.**

In [164]:
playstore[['App','Installs','Price']].head(3)

,App,Installs,Price
2544,Facebook,1000000000,0
336,WhatsApp Messenger,1000000000,0
2604,Instagram,1000000000,0


Have a look on 'Price':

In [165]:
set(playstore['Price'])

{'$0.99',
 '$1.00',
 '$1.04',
 '$1.20',
 '$1.26',
 '$1.29',
 '$1.49',
 '$1.50',
 '$1.59',
 '$1.61',
 '$1.70',
 '$1.75',
 '$1.76',
 '$1.96',
 '$1.97',
 '$1.99',
 '$10.00',
 '$10.99',
 '$109.99',
 '$11.99',
 '$12.99',
 '$13.99',
 '$14.00',
 '$14.99',
 '$15.46',
 '$15.99',
 '$154.99',
 '$16.99',
 '$17.99',
 '$18.99',
 '$19.40',
 '$19.90',
 '$19.99',
 '$2.00',
 '$2.49',
 '$2.50',
 '$2.56',
 '$2.59',
 '$2.60',
 '$2.90',
 '$2.95',
 '$2.99',
 '$200.00',
 '$24.99',
 '$25.99',
 '$28.99',
 '$29.99',
 '$299.99',
 '$3.02',
 '$3.04',
 '$3.08',
 '$3.28',
 '$3.49',
 '$3.61',
 '$3.88',
 '$3.90',
 '$3.95',
 '$3.99',
 '$30.99',
 '$33.99',
 '$37.99',
 '$379.99',
 '$389.99',
 '$39.99',
 '$394.99',
 '$399.99',
 '$4.29',
 '$4.49',
 '$4.59',
 '$4.60',
 '$4.77',
 '$4.80',
 '$4.84',
 '$4.85',
 '$4.99',
 '$400.00',
 '$46.99',
 '$5.00',
 '$5.49',
 '$5.99',
 '$6.49',
 '$6.99',
 '$7.49',
 '$7.99',
 '$74.99',
 '$79.99',
 '$8.49',
 '$8.99',
 '$89.99',
 '$9.00',
 '$9.99',
 '0'}

It is necessary to convert 'Price' to number:

In [166]:
price_to_num = re.compile('\D*(?P<price>\d+\.{0,1}\d*)')
def to_float (price):
    money = price_to_num.search(price)
    if money:
        return float(money.group('price'))
    else:
        return np.nan
playstore['Price'] = playstore.Price.apply(to_float)

In [167]:
set(playstore['Price'])

{0.0,
 0.99,
 1.0,
 1.04,
 1.2,
 1.26,
 1.29,
 1.49,
 1.5,
 1.59,
 1.61,
 1.7,
 1.75,
 1.76,
 1.96,
 1.97,
 1.99,
 2.0,
 2.49,
 2.5,
 2.56,
 2.59,
 2.6,
 2.9,
 2.95,
 2.99,
 3.02,
 3.04,
 3.08,
 3.28,
 3.49,
 3.61,
 3.88,
 3.9,
 3.95,
 3.99,
 4.29,
 4.49,
 4.59,
 4.6,
 4.77,
 4.8,
 4.84,
 4.85,
 4.99,
 5.0,
 5.49,
 5.99,
 6.49,
 6.99,
 7.49,
 7.99,
 8.49,
 8.99,
 9.0,
 9.99,
 10.0,
 10.99,
 11.99,
 12.99,
 13.99,
 14.0,
 14.99,
 15.46,
 15.99,
 16.99,
 17.99,
 18.99,
 19.4,
 19.9,
 19.99,
 24.99,
 25.99,
 28.99,
 29.99,
 30.99,
 33.99,
 37.99,
 39.99,
 46.99,
 74.99,
 79.99,
 89.99,
 109.99,
 154.99,
 200.0,
 299.99,
 379.99,
 389.99,
 394.99,
 399.99,
 400.0}

Create a new column 'Income' and compute the aggregated value for each App, sorted by total income:

In [168]:
playstore['Income']=playstore['Installs']*playstore['Price']
pd.DataFrame(playstore.groupby('App')['Income'].sum().sort_values(ascending=False))

,Income
App,
Minecraft,69900000.0
I am rich,39999000.0
I Am Rich Premium,19999500.0
Hitman Sniper,9900000.0
Grand Theft Auto: San Andreas,6990000.0
Sleep as Android Unlock,5990000.0
Facetune - For Free,5990000.0
DraStic DS Emulator,4990000.0
I'm Rich - Trump Edition,4000000.0


# **12) For each app, compute its minimum and maximum Sentiment_polarity**

First, we have to check if 'Sentiment_Polarity' is composed by numbers or strings

In [42]:
playstore_users['Sentiment_Polarity'].dtype

dtype('float64')

So we can computer min and max.

I create two new columns and print them toghether:

In [43]:
MinPerApp = pd.DataFrame(playstore_users.groupby('App')['Sentiment_Polarity'].min())
MaxPerApp = pd.DataFrame(playstore_users.groupby('App')['Sentiment_Polarity'].max())
MinMax = MinPerApp.merge(MaxPerApp, on = 'App',suffixes=['_min','_max'])
MinMax

,Sentiment_Polarity_min,Sentiment_Polarity_max
App,,
10 Best Foods for You,-0.800000,1.000000
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,-0.112500,0.910000
11st,-1.000000,1.000000
1800 Contacts - Lens Store,-0.300000,0.838542
1LINE – One Line with One Touch,-0.825000,1.000000
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,-0.800000,1.000000
21-Day Meditation Experience,-0.265625,0.587500
"2Date Dating App, Love and matching",-0.645833,1.000000
2GIS: directory & navigator,-0.375000,1.000000


**Print the total execution time:**

In [44]:
print("The total execution time is:",time.time()-start)

The total execution time is: 4.139536142349243
